# 00 · RecSys Runbook — пульт управления 🚦

Цель: единое место, где:
- выбираем профиль/модель/параметры;
- смотрим статус артефактов и fingerprints;
- запускаем стадии пайплайна по кнопкам или целиком;
- проверяем кандидатов/фичи/модели/бленд/реранк;
- собираем сабмит.

**Анти-лик:** все инструменты работают строго time-safe (никаких `.fit` на вал/тест).  
**Паник-режим:** последние 20–40 минут — быстрый конвейер (см. ниже).

In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [2]:
# Базовые импорты
import os, sys, json, time, shlex, glob, gc, math, pathlib, platform, subprocess
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

# Виджеты и отображение
from IPython.display import display, HTML, clear_output
import ipywidgets as W

BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_WRAP = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_THREE_COLS = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")


# Настройка путей
REPO = Path.cwd()
sys.path.insert(0, str(REPO))

print("Python:", sys.version)
print("Platform:", platform.platform())
print("Repo:", REPO)

# Вспомогательные функции: запуск команд с потоковым логом
def run_cmd(cmd_list, env=None, check=True):
    """
    Выполняет команду (list[str]) с потоковым выводом в ячейку.
    Возвращает код возврата.
    """
    print("▶", " ".join(shlex.quote(x) for x in cmd_list))
    process = subprocess.Popen(
        cmd_list, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env
        # bufsize=1  # (по умолчанию норм)
    )
    try:
        for line in iter(process.stdout.readline, ''):
            print(line, end='')
        process.stdout.close()
        ret = process.wait()
    except KeyboardInterrupt:
        process.terminate()
        ret = process.wait()
        print("[aborted]")
    if check and ret != 0:
        raise RuntimeError(f"Command failed with code {ret}: {' '.join(cmd_list)}")
    return ret

def run_tool(script_rel_path, **kwargs):
    """
    Запускает скрипт из репозитория с передачей аргументов вида --key value.
    script_rel_path: например "recsys/tools/run_ranker.py" или "tools/config_show.py"
    """
    args = []
    for k, v in kwargs.items():
        if v is None:
            continue
        if isinstance(v, bool):
            v = "1" if v else "0"
        args += [f"--{k}", str(v)]
    cmd = ["python", script_rel_path] + args
    return run_cmd(cmd)

def mem_gb():
    try:
        import psutil
        return psutil.Process().memory_info().rss / (1024**3)
    except Exception:
        return float('nan')

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Repo: /content


In [ ]:
# Глобальное состояние (заполняется виджетами)
STATE = dict(
    DATASET_ID = "s5e11",
    DATA_PATH  = "/kaggle/input/hear-me-personalized-music-recommender",  # <-- НОВОЕ ПОЛЕ
    PROFILE_PATH = "/kaggle/working/vseros-toolkit/configs/recsys/profiles/gate.yaml",
    MODEL_YAML   = "/kaggle/working/vseros-toolkit/configs/recsys/models/lgbm_ranker.yaml",
    RERANK_YAML  = "/kaggle/working/vseros-toolkit/configs/recsys/rerank.yaml",
    SEED   = 42,
    JOBS   = -1,
    RESUME = True,
    DRY_RUN = False,
    K_SUBMIT = 100,
)

def profile_name_from_path(path):
    return Path(path).stem

def now_tag():
    return datetime.now().strftime("%m%d_%H%M")

def artifacts_paths(dataset_id, profile_path):
    prof = profile_name_from_path(profile_path)
    root = Path("artifacts")
    return dict(
        dataio_dir = root / "recsys" / "dataio" / dataset_id,
        features_dir = root / "recsys" / "features" / dataset_id / prof,
        rerank_dir = root / "recsys" / "rerank" / dataset_id / prof,
        eval_dir = root / "recsys" / "eval" / dataset_id / prof,
        models_dir = root / "recsys" / "models",
        submits_dir = root / "submits" / dataset_id,
        candidates_dir = root / "recsys" / "candidates" / dataset_id / prof,
        dataio_profile_dir = root / "recsys" / "dataio" / dataset_id / prof,
    )

def exists(p): 
    try:
        return Path(p).exists()
    except Exception:
        return False

In [ ]:
# Виджеты параметров
w_dataset   = W.Text(value=STATE["DATASET_ID"], description="dataset_id:", layout=W.Layout(width="300px"))

# НОВЫЙ ВИДЖЕТ ДЛЯ ПУТИ
w_data_path = W.Text(
    value=STATE["DATA_PATH"],
    description="data_path:",
    layout=W.Layout(width="600px")
)

w_profile = W.Text(value=STATE["PROFILE_PATH"], description="profile:", layout=W.Layout(width="500px"))
w_model   = W.Text(value=STATE["MODEL_YAML"],   description="model:",   layout=W.Layout(width="500px"))
w_rerank  = W.Text(value=STATE["RERANK_YAML"],  description="rerank:",  layout=W.Layout(width="500px"))

w_seed    = W.IntText(value=STATE["SEED"], description="seed:", layout=W.Layout(width="150px"))
w_jobs    = W.IntText(value=STATE["JOBS"], description="jobs:", layout=W.Layout(width="150px"))
w_resume  = W.Checkbox(value=STATE["RESUME"], description="resume")
w_dryrun  = W.Checkbox(value=STATE["DRY_RUN"], description="dry-run")
w_k       = W.IntText(value=STATE["K_SUBMIT"], description="topK submit:", layout=W.Layout(width="180px"))

btn_apply = W.Button(description="Применить параметры", button_style="success",
                     layout=BTN_LAYOUT, tooltip="Применить параметры")
out_apply = W.Output()

def on_apply_clicked(_):
    with out_apply:
        clear_output()
        STATE["DATASET_ID"]   = w_dataset.value.strip()
        STATE["DATA_PATH"]    = w_data_path.value.strip()          # <-- Сохраняем путь
        STATE["PROFILE_PATH"] = w_profile.value.strip()
        STATE["MODEL_YAML"]   = w_model.value.strip()
        STATE["RERANK_YAML"]  = w_rerank.value.strip()
        STATE["SEED"]         = int(w_seed.value)
        STATE["JOBS"]         = int(w_jobs.value)
        STATE["RESUME"]       = bool(w_resume.value)
        STATE["DRY_RUN"]      = bool(w_dryrun.value)
        STATE["K_SUBMIT"]     = int(w_k.value)

        # КЛЮЧЕВОЕ: переопределяем paths.data через ENV,
        # чтобы load_config взял именно этот путь
        os.environ["CFG_paths__data"] = STATE["DATA_PATH"]

        print("✓ Параметры обновлены:")
        print(json.dumps(STATE, ensure_ascii=False, indent=2))

btn_apply.on_click(on_apply_clicked)

W.VBox([
    W.HTML("<h3>Параметры тура</h3>"),
    W.HBox([w_dataset, w_seed, w_jobs, w_k, w_resume, w_dryrun], layout=ROW_WRAP),
    w_data_path,   # <-- показываем поле пути
    w_profile,
    w_model,
    w_rerank,
    btn_apply,
    out_apply
])

In [5]:
btn_show_cfg = W.Button(description="Показать resolved config", button_style="", layout=BTN_LAYOUT, tooltip="Показать resolved config")
btn_fprint   = W.Button(description="Fingerprint профиля", button_style="", layout=BTN_LAYOUT, tooltip="Fingerprint профиля")
out_cfg      = W.Output()
out_fp       = W.Output()

def on_show_cfg(_):
    with out_cfg:
        clear_output()
        run_tool("tools/config_show.py",
                 subsystem="recsys",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 resolve=1, **{"as":"yaml"})

def on_fp(_):
    with out_fp:
        clear_output()
        run_tool("tools/config_fingerprint.py",
                 subsystem="recsys",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"])

btn_show_cfg.on_click(on_show_cfg)
btn_fprint.on_click(on_fp)

W.VBox([
    W.HTML("<h3>Конфиги и fingerprint</h3>"),
    W.HBox([btn_show_cfg, btn_fprint], layout=ROW_WRAP),
    out_cfg,
    out_fp
])

In [6]:
btn_refresh = W.Button(description="Обновить статус", button_style="info", layout=BTN_LAYOUT, tooltip="Обновить статус")
out_status  = W.Output()

def collect_status():
    A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
    prof = profile_name_from_path(STATE["PROFILE_PATH"])
    ddir = A["dataio_dir"]; fdir = A["features_dir"]; rdir = A["rerank_dir"]
    edir = A["eval_dir"];    mdir = A["models_dir"];   cdir = A["candidates_dir"]

    pairs_train = A["dataio_profile_dir"] / "pairs_train.parquet"
    pairs_val   = A["dataio_profile_dir"] / "pairs_val.parquet"
    pairs_test  = A["dataio_profile_dir"] / "pairs_test.parquet"

    rows = []
    rows += [("dataio/interactions_norm", (ddir/"interactions_norm.parquet"))]
    rows += [("dataio/items_norm",        (ddir/"items_norm.parquet"))]
    rows += [("dataio/splits.json",       (ddir/"splits.json"))]
    rows += [("dataio/queries_val",       (A["dataio_profile_dir"]/"queries_val.parquet"))]
    rows += [("dataio/pairs_train",       pairs_train)]
    rows += [("dataio/pairs_val",         pairs_val)]
    rows += [("dataio/pairs_test",        pairs_test)]
    rows += [("candidates/candidates",    (cdir/"candidates.parquet"))]
    rows += [("features/X_train",         (fdir/"X_train.parquet"))]
    rows += [("features/X_test",          (fdir/"X_test.parquet"))]
    # последний ModelRun (по времени)
    try:
        all_runs = sorted(Path(mdir).glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)
        last_run = all_runs[0] if all_runs else None
    except Exception:
        last_run = None
    rows += [("model/last_run", last_run/"meta.json" if last_run else Path("__none__"))]
    rows += [("rerank/recommendations",   (rdir/"recommendations.parquet"))]
    rows += [("eval/report",              (edir/"eval_report.json"))]
    rows += [("submit/any_csv",           (A["submits_dir"]))]

    data = []
    for name, path in rows:
        path = Path(path)
        ok = path.exists() if (path.suffix or path.is_file()) else (path.exists() and any(path.glob("*.csv")))
        mtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(path.stat().st_mtime)) if path.exists() else "-"
        data.append((name, str(path), "OK" if ok else "MISSING", mtime))
    df = pd.DataFrame(data, columns=["artifact","path","status","modified"])
    return df

def style_status(df):
    def colorize(val):
        if val == "OK":
            return 'background-color: #c7f6c7'
        return 'background-color: #ffd6d6'
    sty = df.style.applymap(colorize, subset=["status"])
    return sty

def on_refresh(_):
    with out_status:
        clear_output()
        df = collect_status()
        display(style_status(df))

btn_refresh.on_click(on_refresh)

W.VBox([
    W.HTML("<h3>Статус артефактов</h3>"),
    btn_refresh,
    out_status
])

In [7]:
# Общая обёртка для сборки аргументов
def _args_common():
    return dict(seed=STATE["SEED"], jobs=STATE["JOBS"])

# --- adapt
btn_adapt = W.Button(description="1) adapt", button_style="primary", layout=BTN_LAYOUT, tooltip="Импортировать и нормализовать данные")
out_adapt = W.Output()

def on_adapt(_):
    with out_adapt:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_tool("recsys/tools/run_adapt.py",
                 dataset_id=STATE["DATASET_ID"],
                 schema="configs/recsys/schema.yaml",
                 data_interactions=f"data/{STATE['DATASET_ID']}/interactions.parquet",
                 data_items=f"data/{STATE['DATASET_ID']}/items.parquet",
                 out_dir=str(A["dataio_dir"]),
                 save_indexers=1)
btn_adapt.on_click(on_adapt)

# --- sessionize
btn_session = W.Button(description="2) sessionize", button_style="primary", layout=BTN_LAYOUT, tooltip="Построить сессии пользователей")
out_session = W.Output()

def on_session(_):
    with out_session:
        clear_output()
        run_tool("recsys/tools/run_sessionize.py",
                 dataset_id=STATE["DATASET_ID"],
                 session_gap_min=30,
                 overwrite=0)
btn_session.on_click(on_session)

# --- splits
btn_splits = W.Button(description="3) splits", button_style="primary", layout=BTN_LAYOUT, tooltip="Создать обучающие и валидационные сплиты")
out_splits = W.Output()

def on_splits(_):
    with out_splits:
        clear_output()
        run_tool("recsys/tools/run_splits.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"])
btn_splits.on_click(on_splits)

# --- build_pairs
btn_pairs = W.Button(description="4) build_pairs", button_style="primary", layout=BTN_LAYOUT, tooltip="Собрать пары для обучения")
out_pairs = W.Output()

def on_pairs(_):
    with out_pairs:
        clear_output()
        run_tool("recsys/tools/run_build_pairs.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 scope="session",
                 neg_pos_ratio=100,
                 neg_strategy="pop")
btn_pairs.on_click(on_pairs)

# --- candidates
btn_cands = W.Button(description="5) candidates", button_style="primary", layout=BTN_LAYOUT, tooltip="Сгенерировать кандидатов")
out_cands = W.Output()

def on_cands(_):
    with out_cands:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_tool("recsys/tools/run_candidates.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 schema="configs/recsys/schema.yaml",
                 data_interactions=str(A["dataio_dir"]/"interactions_norm.parquet"),
                 data_items=str(A["dataio_dir"]/"items_norm.parquet"),
                 data_queries=str(A["dataio_profile_dir"]/"queries_val.parquet"),
                 K=200, jobs=STATE["JOBS"], seed=STATE["SEED"], cache=1)
btn_cands.on_click(on_cands)

# --- features
btn_feats = W.Button(description="6) features", button_style="primary", layout=BTN_LAYOUT, tooltip="Построить фичи")
out_feats = W.Output()

def on_feats(_):
    with out_feats:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_tool("recsys/tools/run_features.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 features="configs/recsys/features.yaml",
                 pairs_train=str(A["dataio_profile_dir"]/"pairs_train.parquet"),
                 pairs_test=str(A["dataio_profile_dir"]/"pairs_test.parquet"),
                 interactions=str(A["dataio_dir"]/"interactions_norm.parquet"),
                 items=str(A["dataio_dir"]/"items_norm.parquet"),
                 out_dir=str(A["features_dir"]),
                 jobs=STATE["JOBS"], seed=STATE["SEED"], cache=1)
btn_feats.on_click(on_feats)

# --- ranker
btn_ranker = W.Button(description="7) ranker", button_style="primary", layout=BTN_LAYOUT, tooltip="Обучить модель ранжирования")
out_ranker = W.Output()

def on_ranker(_):
    with out_ranker:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        run_tool("recsys/tools/run_ranker.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 model=STATE["MODEL_YAML"],
                 X_train=str(A["features_dir"]/"X_train.parquet"),
                 X_test=str(A["features_dir"]/"X_test.parquet"),
                 pairs_train=str(A["dataio_profile_dir"]/"pairs_train.parquet"),
                 pairs_test=str(A["dataio_profile_dir"]/"pairs_test.parquet"),
                 out_dir=str(A["models_dir"]),
                 seed=STATE["SEED"], jobs=STATE["JOBS"], resume=1 if STATE["RESUME"] else 0)
btn_ranker.on_click(on_ranker)

# --- blend (по желанию)
btn_blend = W.Button(description="8) blend (optional)", button_style="", layout=BTN_LAYOUT, tooltip="Собрать бленд нескольких ModelRun")
out_blend = W.Output()

def on_blend(_):
    with out_blend:
        clear_output()
        mdir = Path("artifacts/recsys/models")
        runs = sorted(mdir.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)[:3]
        if not runs:
            print("Нет ModelRun для бленда")
            return
        run_list = ",".join(str(p) for p in runs)
        run_tool("recsys/tools/run_blend.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 runs=run_list,
                 mode="nnls",
                 out_dir=str(mdir),
                 seed=STATE["SEED"])
btn_blend.on_click(on_blend)

# --- rerank
btn_rerank = W.Button(description="9) rerank", button_style="warning", layout=BTN_LAYOUT, tooltip="Пересортировать рекомендации")
out_rerank = W.Output()

def on_rerank(_):
    with out_rerank:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        # Найти последний run (или blended)
        mdir = Path("artifacts/recsys/models")
        runs = sorted(mdir.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)
        if not runs:
            print("ModelRun не найден")
            return
        last = runs[0]
        test_pred = last / "test_pred.npy"
        if not test_pred.exists():
            print("test_pred.npy отсутствует у последнего ран-а. Обучи модель.")
            return
        run_tool("recsys/tools/run_rerank.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 rerank=STATE["RERANK_YAML"],
                 pairs_test=str(A["dataio_profile_dir"]/"pairs_test.parquet"),
                 preds_test=str(test_pred),
                 items=str(A["dataio_dir"]/"items_norm.parquet"),
                 K=STATE["K_SUBMIT"],
                 out_dir=str(A["rerank_dir"]))
btn_rerank.on_click(on_rerank)

# --- eval
btn_eval = W.Button(description="10) eval", button_style="warning", layout=BTN_LAYOUT, tooltip="Посчитать метрики и отчёт срезов")
out_eval = W.Output()

def on_eval(_):
    with out_eval:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        recos = A["rerank_dir"] / "recommendations.parquet"
        if not recos.exists():
            print("recommendations.parquet не найден — сначала запусти rerank")
            return
        run_tool("recsys/tools/run_eval.py",
                 pairs=str(A["dataio_profile_dir"]/"pairs_val.parquet"),
                 preds=str(recos),
                 metrics="recall@20,ndcg@20,coverage@20,novelty@20,diversity@20",
                 slices="brand,category",
                 bootstrap=200,
                 out=str(A["eval_dir"]/"eval_report.json"))
btn_eval.on_click(on_eval)

# --- submit
btn_submit = W.Button(description="11) submit", button_style="danger", layout=BTN_LAYOUT, tooltip="Собрать сабмит в Kaggle-формате")
out_submit = W.Output()

def on_submit(_):
    with out_submit:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        Path(A["submits_dir"]).mkdir(parents=True, exist_ok=True)
        out_csv = A["submits_dir"] / f"{now_tag()}_{profile_name_from_path(STATE['PROFILE_PATH'])}.csv"
        recos = A["rerank_dir"] / "recommendations.parquet"
        if not recos.exists():
            print("recommendations.parquet не найден — сначала запусти rerank")
            return
        run_tool("recsys/tools/run_submit.py",
                 dataset_id=STATE["DATASET_ID"],
                 recommendations=str(recos),
                 format="kaggle",
                 topk=STATE["K_SUBMIT"],
                 out=str(out_csv),
                 tag=f"{STATE['DATASET_ID']}_{profile_name_from_path(STATE['PROFILE_PATH'])}_{now_tag()}")
btn_submit.on_click(on_submit)

stage_buttons = [
    btn_adapt, btn_session, btn_splits, btn_pairs,
    btn_cands, btn_feats, btn_ranker, btn_blend,
    btn_rerank, btn_eval, btn_submit,
]
stage_grid = W.GridBox(children=stage_buttons, layout=GRID_THREE_COLS)

W.VBox([
    W.HTML("<h3>Запуск стадий</h3>"),
    stage_grid,
    out_adapt, out_session, out_splits, out_pairs,
    out_cands, out_feats, out_ranker, out_blend,
    out_rerank, out_eval, out_submit
])


In [8]:
btn_full = W.Button(description="▶ Полный пайплайн (gate)", button_style="success", layout=BTN_LAYOUT, tooltip="Запустить полный пайплайн gate")
btn_panic= W.Button(description="⚡ Паник-пайплайн (20–40 мин)", button_style="danger", layout=BTN_LAYOUT, tooltip="Быстрый минимальный паник-пайплайн")
out_pipe = W.Output()

def on_full(_):
    with out_pipe:
        clear_output()
        stages = "adapt,sessionize,splits,build_pairs,candidates,features,ranker,rerank,eval"
        run_tool("recsys/tools/run_pipeline.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile=STATE["PROFILE_PATH"],
                 stages=stages,
                 resume=1 if STATE["RESUME"] else 0,
                 seed=STATE["SEED"],
                 jobs=STATE["JOBS"])

def on_panic(_):
    with out_pipe:
        clear_output()
        # минимальный профиль "panic" (должен быть в configs/recsys/profiles/panic.yaml)
        stages = "build_pairs,candidates,features,ranker,rerank,submit"
        run_tool("recsys/tools/run_pipeline.py",
                 dataset_id=STATE["DATASET_ID"],
                 profile="configs/recsys/profiles/panic.yaml",
                 stages=stages,
                 resume=1, seed=STATE["SEED"], jobs=STATE["JOBS"])

btn_full.on_click(on_full)
btn_panic.on_click(on_panic)

W.VBox([
    W.HTML("<h3>Оркестратор</h3>"),
    W.HBox([btn_full, btn_panic], layout=ROW_WRAP),
    out_pipe
])


In [9]:
btn_rec_recall = W.Button(description="Retrieval Recall@20 (быстрый чек)", button_style="", layout=BTN_LAYOUT, tooltip="Быстрый hit-rate по candidates.parquet")
out_rec_recall = W.Output()

def on_rec_recall(_):
    from collections import defaultdict
    with out_rec_recall:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        pairs_val = A["dataio_profile_dir"] / "pairs_val.parquet"
        cands     = A["candidates_dir"] / "candidates.parquet"
        if not pairs_val.exists() or not cands.exists():
            print("Нет pairs_val или candidates. Запусти build_pairs и candidates.")
            return
        dfp = pd.read_parquet(pairs_val)
        dfc = pd.read_parquet(cands)
        k = 20
        dfc = dfc.sort_values(["query_id","score"], ascending=[True,False]).groupby("query_id").head(k)
        true_items = dfp[dfp["label"]==1].groupby("query_id")["item_id"].apply(set).to_dict()
        pred_items = dfc.groupby("query_id")["item_id"].apply(list).to_dict()
        ok, tot = 0, 0
        for q, ts in true_items.items():
            pi = pred_items.get(q, [])
            hit = int(any(it in ts for it in pi))
            ok += hit; tot += 1
        recall_at_k = ok / max(1, tot)
        print(f"Retrieval HitRate/Recall@{k}: {recall_at_k:.4f}  (queries={tot})")

btn_rec_recall.on_click(on_rec_recall)

W.VBox([
    W.HTML("<h3>Быстрые проверки</h3>"),
    btn_rec_recall,
    out_rec_recall
])


In [10]:
btn_feat_passport = W.Button(description="Паспорт фичей (catalog.json)", button_style="", layout=BTN_LAYOUT, tooltip="Показать содержимое catalog.json")
out_feat_passport = W.Output()

def on_feat_passport(_):
    with out_feat_passport:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        cat = A["features_dir"] / "catalog.json"
        if not cat.exists():
            print("catalog.json не найден — запусти features")
            return
        C = json.loads(cat.read_text())
        print("Всего фич:", C.get("total_features"))
        if "blocks" in C:
            dfb = pd.DataFrame(C["blocks"])
            display(dfb)

btn_feat_passport.on_click(on_feat_passport)
W.VBox([btn_feat_passport, out_feat_passport])


In [11]:
btn_model_summary = W.Button(description="Сводка ModelRun", button_style="", layout=BTN_LAYOUT, tooltip="Сводка ModelRun")
out_model_summary = W.Output()

def on_model_summary(_):
    with out_model_summary:
        clear_output()
        mdir = Path("artifacts/recsys/models")
        runs = sorted(mdir.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)
        if not runs:
            print("ModelRun не найден")
            return
        last = runs[0]
        meta_p = last / "meta.json"
        if not meta_p.exists():
            print("meta.json отсутствует:", last)
            return
        meta = json.loads(meta_p.read_text())
        print(json.dumps({k:meta.get(k) for k in ["run_id","backend","task","cv_mean","cv_std"]}, ensure_ascii=False, indent=2))
        fi_p = last / "feature_importance.csv"
        if fi_p.exists():
            fi = pd.read_csv(fi_p)
            display(fi.sort_values("importance", ascending=False).head(20))
        else:
            print("feature_importance.csv отсутствует")

btn_model_summary.on_click(on_model_summary)
W.VBox([btn_model_summary, out_model_summary])

In [13]:
btn_eval_view = W.Button(description="Показать eval_report.json", button_style="", layout=BTN_LAYOUT, tooltip="Показать eval_report.json")
out_eval_view = W.Output()

def on_eval_view(_):
    with out_eval_view:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        rep = A["eval_dir"] / "eval_report.json"
        if not rep.exists():
            print("eval_report.json не найден — запусти eval")
            return
        R = json.loads(rep.read_text())
        overall = R.get("overall", {})
        print("OVERALL metrics:")
        for k,v in overall.items():
            if isinstance(v, (int,float)):
                print(f"  {k}: {v:.6f}")
            else:
                print(f"  {k}: {v}")
        if "slices" in R:
            print("SLICES (первые 10):")
            items = list(R["slices"].items())[:10]
            df = pd.DataFrame([{"slice":k, **vals} for k,vals in items])
            display(df)

btn_eval_view.on_click(on_eval_view)
W.VBox([btn_eval_view, out_eval_view])

In [14]:
btn_list_sub = W.Button(description="Показать сабмиты", button_style="", layout=BTN_LAYOUT, tooltip="Показать сабмиты")
btn_list_logs= W.Button(description="Показать артефакты (дерево)", button_style="", layout=BTN_LAYOUT, tooltip="Показать артефакты")
out_misc     = W.Output()

def on_list_sub(_):
    with out_misc:
        clear_output()
        A = artifacts_paths(STATE["DATASET_ID"], STATE["PROFILE_PATH"])
        subdir = A["submits_dir"]
        if not subdir.exists():
            print("Папка сабмитов пуста")
            return
        files = sorted(subdir.glob("*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
        df = pd.DataFrame([{"file":f.name, "size_MB": round(f.stat().st_size/1024/1024,2),
                            "modified": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(f.stat().st_mtime))}
                           for f in files])
        display(df)

def on_list_logs(_):
    with out_misc:
        clear_output()
        rows = []
        for base in ["artifacts/recsys", "artifacts/submits"]:
            for p in sorted(Path(base).rglob("*"))[-400:]:
                try:
                    rows.append(str(p))
                except Exception:
                    pass
        print("".join(rows[-200:]))

btn_list_sub.on_click(on_list_sub)
btn_list_logs.on_click(on_list_logs)

W.VBox([
    W.HBox([btn_list_sub, btn_list_logs], layout=ROW_WRAP),
    out_misc
])


In [ ]:
w_deadline = W.Text(value="23:59", description="Дедлайн (HH:MM):", layout=W.Layout(width="220px"))
btn_left   = W.Button(description="Сколько осталось?", button_style="", layout=BTN_LAYOUT, tooltip="Сколько осталось?")
out_time   = W.Output()

def on_left(_):
    with out_time:
        clear_output()
        try:
            hh, mm = map(int, w_deadline.value.strip().split(":"))
            now = datetime.now()
            target = now.replace(hour=hh, minute=mm, second=0, microsecond=0)
            if target < now:
                target = target.replace(day=now.day)  # примитивно: в те же сутки
            dt = target - now
            mins = dt.total_seconds() / 60
            print(f"До дедлайна ~ {int(mins)} мин")
        except Exception as e:
            print("Не получилось разобрать время:", e)

btn_left.on_click(on_left)
W.VBox([W.HTML("<h3>Таймер</h3>"), W.HBox([w_deadline, btn_left], layout=ROW_WRAP), out_time])

### Частые ошибки и быстрые фиксы

- **Низкий Recall@20 у кандидатов** → увеличь окно/добавь генераторы/подними квоты (`02_candidates_lab` / профили).
- **Несовпадение колонок X_train/X_test** → очисти кэш features, проверь профили блоков, снова `run_features`.
- **Time-guard сработал** → проверь `ts_query` в `pairs_{split}`, эмбарго/границы и параметры окон в features.
- **model не даёт прироста** → проверь важности, убери шумные блоки, добавь sequence/similarity, покрути `ndcg_k`.
- **eval странный** → смотри `slices` (категории/бренды/время) и bootstrap интервалы.